# Link Prediction in Condmat

In [2]:
import copy
import datetime
import itertools
import math
from typing import List, Any, Dict, Tuple

import joblib
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import precision_recall_curve, average_precision_score, roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold, cross_validate, GridSearchCV, train_test_split
import seaborn as sns
from tqdm import tqdm
from xgboost import XGBClassifier

# Typing
NodePair = Tuple[int, int]
Edge = List[Tuple[int, int, Dict[str, datetime.datetime]]]

folder = '/local/bruingjde/complexnetworks2020-experiment/temp/en-a1'

In [9]:
def filter_edgelist(edges, *, start=None, stop=None) -> List[Edge]: 
  """Filter edgelist.  If start/ stop is float, start/stop from the fraction of total edges. If datetime, this is used.""" 
  no_edges = len(edges)
  if start is None: start=0
  if stop is None: stop=1
  if type(start) is float or start == 0:
    start_index = int(start*no_edges)
    start = edges[start_index][2]['date']
  if type(stop) is float or stop == 1:
    stop_index = math.floor(stop*no_edges)-1
    stop = edges[stop_index][2]['date']
  return [edge for edge in edges if edge[2]['date'] >= start and edge[2]['date'] <= stop]
def import_file(file='src/dg/out.munmun_digg_reply'):
  d = pd.read_csv(file, sep=' ', skiprows=1, names=['source', 'target', 'weight', 'date'])
  d['date'] = d['date'].apply(datetime.datetime.fromtimestamp)
  return [(u, v, dict(date=date)) for u, v, date in d.sort_values('date')[['source', 'target', 'date']].itertuples(index=False)]
def giant_component(graph: nx.Graph) -> nx.Graph: return graph.subgraph(max(nx.connected_components(graph), key=len)).copy()
def get_graph(edgelist: List[Edge]) -> nx.Graph:
  """Add edge to graph. Contains edge attribute weight."""
  g = nx.Graph()
  
  for u, v, _ in edgelist:
    weight = g[u][v]["weight"]+1 if g.has_edge(u,v) else 1
    g.add_edge(u, v, weight=weight)
  
  return g
def report(graph:nx.Graph, probes: Tuple[int, int]):
  n = len(probes)
  print(f"Number of probes: {n}")
  a = sum([graph.has_edge(u, v) for u, v in probes])
  print(f"- already edge: {a} ({a/n:.0%})")
  non_edges = set(nx.non_edges(graph))
  ne = sum([np in non_edges for np in tqdm(probes)])
  print(f"- both nodes in graph: {ne} ({ne/n:.0%})")
  ng = sum([not (graph.has_node(u) and graph.has_node(v)) for u, v in tqdm(probes)])
  print(f"- not in graph: {ng} ({ng/n:.0%})")
def get_distances(graph: nx.Graph, cutoff: int = None) -> (List[NodePair], List[int]):
  """
  Get all non-edges using BFS. When cutoff provided, consider only node pairs with at most this distance.
  Returns:
  - nodepairs: tuple containing all nodepairs
  - distances: tuple containing all distances
  """
  return zip(
    *[
      [(u, v), distance]
      for u, (nbs_u, _) in tqdm(nx.all_pairs_dijkstra(graph, cutoff, weight=None), total=len(graph), desc="get_distances")
      for v, distance in nbs_u.items() if distance > 1 and (cutoff is None or distance <= cutoff) 
    ]
  )

In [7]:
edges = import_file()
joblib.dump(edges, 'src/dg/edges.pkl')

['src/dg/edges.pkl']

## Set-up
Choose here the parameters on how you want to define the learn and assessing phase.

In [10]:
g_learn = giant_component(get_graph(filter_edgelist(edges, stop=.7)))
uv_assessing = {(u, v) for u, v, _ in filter_edgelist(edges, start=.7)}

In [83]:
joblib.dump(g_learn, f'{folder}/graph.pkl')
joblib.dump(uv_assessing, f'{folder}/probes.pkl')

['/local/bruingjde/complexnetworks2020-experiment/temp/en-a1//probes.pkl']

In [11]:
report(graph=g_learn, probes=uv_assessing)

Number of probes: 26048
- already edge: 196 (1%)


100%|██████████| 26048/26048 [00:00<00:00, 1270874.06it/s]

- both nodes in graph: 7970 (31%)
- not in graph: 10576 (41%)


## Export

In [ ]:
%%time
targets = joblib.load(f'{folder}all/targets.pkl')
nodepairs = joblib.load(f'{folder}all/nodepairs.pkl')
distances = joblib.load(f'{folder}all/distances.pkl')
g_learn = joblib.load(f'{folder}all/graph.pkl')

In [12]:
nodepairs, distances = get_distances(g_learn, cutoff=3)
targets = [nodepair in uv_assessing for nodepair in tqdm(nodepairs)]

100%|██████████| 35970964/35970964 [00:10<00:00, 3500977.09it/s]


In [ ]:
%%time
nodepairs = np.array(nodepairs)
distances = np.array(distances)
targets = np.array(targets)
for select_distance in [3]:
  print(select_distance)
  filter_indices = (distances == select_distance)
  nodepairs[filter_indices].dump(f'{folder}{select_distance}/nodepairs.pkl')
  distances[filter_indices].dump(f'{folder}{select_distance}/distances.pkl')
  targets[filter_indices].dump(f'{folder}{select_distance}/targets.pkl')
  joblib.dump(g_learn, f'{folder}{select_distance}/graph.pkl')

In [18]:
print(f'{sum(targets) / len(nodepairs):e}')

2.935012e-05


## Distance analysis

In [ ]:
df = pd.DataFrame(dict(distances=distances, targets=targets))
df = df.groupby('distances')['targets'].agg(['size', 'sum'])

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['size'], fill='tozeroy', name='# Nodepairs'))
fig.add_trace(go.Scatter(x=df.index, y=df['sum'], fill='tozeroy', name='# Positives'))
fig.update_layout(xaxis=dict(tickmode='linear', tick0=2, dtick=1), yaxis_type="log")
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['size'].cumsum(), fill='tozeroy', name='# Nodepairs'))
fig.add_trace(go.Scatter(x=df.index, y=df['sum'].cumsum(), fill='tozeroy', name='# Positives'))
fig.update_layout(xaxis=dict(tickmode='linear', tick0=2, dtick=1), yaxis_type="log")

## Feature inspection

In [ ]:
%%time
df = joblib.load("temp/a4n/2/features.pkl")

In [ ]:
fig = px.imshow(df.corr(), x=df.columns, y=df.columns)
fig.update_xaxes(side="top")

In [ ]:
def pairplot(df):
  return sns.pairplot(
    df[df['target']].append(df[~df['target']].sample(sum(df['target']))).apply(minmax_scale), 
    hue='target',
    hue_order=[True, False],
    palette={True: 'green', False: 'red'},
    kind='reg',
    diag_kws=dict(bw=.02),
    plot_kws=dict(scatter_kws=dict(alpha=.1))
  )

In [ ]:
pairplot(df)

## Hyperparameter selection

See `parameter_optimalization.ipynb`. We choose the following parameters:
- `max_depth = 1`
- `tree_method = 'hist'`
- `no feature scaling`

### XGBoost

$n=2$

In [ ]:
def gridsearch(df: pd.DataFrame, random_state=1, also_random=True, max_depth=[1, 2]) -> pd.DataFrame:
  X = df.drop(columns='target').values
  y = df['target'].values
  
  param_grid=dict(max_depth=max_depth, scale_pos_weight=[sum(~y)/sum(y), 1])
  
  X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=1/3, random_state=random_state)
  clf = XGBClassifier(random_state=random_state, tree_method='hist', n_jobs=6)
  gridsearch = GridSearchCV(
    clf, 
    param_grid=param_grid, 
    scoring='average_precision', 
    n_jobs=30,
    cv=StratifiedKFold(shuffle=True, random_state=random_state),
    return_train_score=True
  )
  
  if also_random: 
    gridsearch_random = copy.deepcopy(gridsearch)
    np.random.seed(random_state)
    y_random = copy.deepcopy(y_trainval)
    np.random.shuffle(y_random)
  
  gridsearch.fit(X_trainval, y_trainval)
  df_dict = dict(
      mean_train=gridsearch.cv_results_['mean_train_score'],
      std_train=gridsearch.cv_results_['std_train_score'],
      mean_test=gridsearch.cv_results_['mean_test_score'],
      std_test=gridsearch.cv_results_['std_test_score'],
      test_fold0=gridsearch.cv_results_[f'split0_test_score'],
      test_fold1=gridsearch.cv_results_[f'split1_test_score'],
      test_fold2=gridsearch.cv_results_[f'split2_test_score'],
      test_fold3=gridsearch.cv_results_[f'split3_test_score'],
      test_fold4=gridsearch.cv_results_[f'split4_test_score']
  )
  
  if also_random: 
    gridsearch_random.fit(X_trainval, y_random)
    df_dict['mean_train_random']=gridsearch_random.cv_results_['mean_train_score']
    df_dict['std_train_random']=gridsearch_random.cv_results_['std_train_score']
    df_dict['mean_test_random']=gridsearch_random.cv_results_['mean_test_score']
    df_dict['std_test_random']=gridsearch_random.cv_results_['std_test_score']
  df = pd.DataFrame(df_dict, index=pd.Index([(d['max_depth'], d['scale_pos_weight'] > 1) for d in gridsearch.cv_results_['params']], name=('max_depth', 'balanced')))
  df['diff_train_test'] = (df['mean_test'] - df['mean_train']).abs()
  df['rstd_test'] = df['std_test'] / df['mean_test']
  if also_random: df['test_over_random'] = df['mean_test'] - df['mean_test_random']
  return df.sort_values('mean_test', ascending=False)
    
def report_performance(df: pd.DataFrame, random_state=1, max_depth=1, tree_method='hist', balanced=True, n_jobs=128):
  X = df.drop(columns='target').values
  y = df['target'].values
  X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=1/3, random_state=random_state)
  clf = XGBClassifier(max_depth=max_depth, n_jobs=128, tree_method=tree_method, scale_pos_weight=sum(~y)/sum(y) if balanced else 1 , random_state=random_state)
  clf.fit(X_trainval, y_trainval)
  y_pred = clf.predict_proba(X_test)[:,1]
  return average_precision_score(y_test, y_pred), roc_auc_score(y_test, y_pred)

In [ ]:
df = pd.read_pickle(f'temp/a1/2/features.pkl')
X = df.drop(columns='target').values
y = df['target'].values

param_grid=dict(max_depth=[1, 2], scale_pos_weight=[sum(~y)/sum(y), 1])

X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=1/3, random_state=1)
clf = XGBClassifier(random_state=1, tree_method='hist', n_jobs=6)
gridsearch = GridSearchCV(
  clf, 
  param_grid=param_grid, 
  scoring='average_precision', 
  n_jobs=30,
  cv=StratifiedKFold(shuffle=True, random_state=1),
  return_train_score=True
)
gridsearch.fit(X_trainval, y_trainval)

In [ ]:
hps2 = gridsearch(pd.read_pickle(f'temp/a1/2/features.pkl'))

In [ ]:
hps2[['mean_test', 'diff_train_test', 'rstd_test', 'test_over_random', 'test_fold0', 'test_fold1', 'test_fold2', 'test_fold3', 'test_fold4']]

In [ ]:
hps3 = gridsearch(pd.read_pickle(f'temp/a1/3/features.pkl'))

In [ ]:
hps3[['mean_test', 'diff_train_test', 'rstd_test', 'test_over_random', 'test_fold0', 'test_fold1', 'test_fold2', 'test_fold3', 'test_fold4']]

In [ ]:
hps4 = gridsearch(pd.read_pickle(f'temp/a1/4/features.pkl'))

In [ ]:
hps4[['mean_test', 'diff_train_test', 'rstd_test', 'test_over_random', 'test_fold0', 'test_fold1', 'test_fold2', 'test_fold3', 'test_fold4']]

In [ ]:
report_performance(pd.read_pickle(f'temp/a1/2/features.pkl'), balanced=False)

In [ ]:
report_performance(pd.read_pickle(f'temp/a1/3/features.pkl'), balanced=True)

In [ ]:
report_performance(pd.read_pickle(f'temp/a1/4/features.pkl'), balanced=True)